## End to end examples logging data to Galileo for Text Classification, MLTC, and NER

### For understanding the client and how to get started, see the [Dataquality Demo](./Dataquality-Client-Demo.ipynb)
### Check out the full documentation [here](https://rungalileo.gitbook.io/galileo/getting-started)
### To see real end-to-end notebooks training real ML models, see [here](https://drive.google.com/drive/folders/17-cHuRzXIpWaD8rYwy69RMQr__HiAiDk?usp=sharing)

In [1]:
## Local

import os

os.environ['GALILEO_CONSOLE_URL']="http://localhost:8088"
os.environ["GALILEO_USERNAME"]="user@example.com"
os.environ["GALILEO_PASSWORD"]="Th3secret_"

In [2]:
import dataquality as dq
dq.configure()

📡 http://localhost:8088
🔭 Logging you into Galileo

👀 Found auth method email set via env, skipping prompt.
🚀 You're logged in to Galileo as user@example.com!


***Helper function***

In [3]:
from dataquality import config
import pandas as pd
from dataquality.clients.api import ApiClient
from time import sleep


api_client = ApiClient()


def see_results(wait=True, body={}):
    if wait:
        print("Waiting for data to be processed")
        api_client.wait_for_run()

    task_type = dq.config.task_type
    proj = api_client.get_project(config.current_project_id)["name"]
    run = api_client.get_project_run(config.current_project_id, config.current_run_id)["name"]
    api_client.export_run(proj, run, "training", f"{task_type}_training.csv")
    api_client.export_run(proj, run, "test", f"{task_type}_test.csv")
    api_client.export_run(proj, run, "validation", f"{task_type}_validation.csv")
    print(f"Exported to {task_type}_training.csv, {task_type}_test.csv, and {task_type}_validation.csv")
    df_train = pd.read_csv(f"{task_type}_training.csv")
    df_test = pd.read_csv(f"{task_type}_test.csv")
    df_val = pd.read_csv(f"{task_type}_validation.csv")
    print("Training")
    display(df_train)
    print("\nTest")
    display(df_test)
    print("\nValidation")
    display(df_val)
    return df_train, df_test, df_val

## NER

In [4]:
from dataquality.schemas.task_type import TaskType
from dataquality import config 
from uuid import uuid4
import numpy as np
from time import sleep
from tqdm.notebook import tqdm


dq.init("text_ner", "test-ner-proj", "test-ner-run")


def log_inputs():
    text_inputs = ['what movies star bruce willis', 'show me films with drew barrymore from the 1980s', 'what movies starred both al pacino and robert deniro', 'find me all of the movies that starred harold ramis and bill murray', 'find me a movie with a quote about baseball in it']
    tokens = [[(0, 4), (5, 11), (12, 16), (17, 22), (17, 22), (23, 29), (23, 29)], [(0, 4), (5, 7), (8, 13), (14, 18), (19, 23), (24, 33), (24, 33), (24, 33), (34, 38), (39, 42), (43, 48)], [(0, 4), (5, 11), (12, 19), (20, 24), (25, 27), (28, 34), (28, 34), (28, 34), (35, 38), (39, 45), (39, 45), (46, 52), (46, 52)], [(0, 4), (5, 7), (8, 11), (12, 14), (15, 18), (19, 25), (26, 30), (31, 38), (39, 45), (39, 45), (39, 45), (46, 51), (46, 51), (52, 55), (56, 60), (61, 67), (61, 67), (61, 67)], [(0, 4), (5, 7), (8, 9), (10, 15), (16, 20), (21, 22), (23, 28), (29, 34), (35, 43), (44, 46), (47, 49)]]
    gold_spans = [[{'start': 17, 'end': 29, 'label': 'ACTOR'}], [{'start': 19, 'end': 33, 'label': 'ACTOR'}, {'start': 43, 'end': 48, 'label': 'YEAR'}], [{'start': 25, 'end': 34, 'label': 'ACTOR'}, {'start': 39, 'end': 52, 'label': 'ACTOR'}], [{'start': 39, 'end': 51, 'label': 'ACTOR'}, {'start': 56, 'end': 67, 'label': 'ACTOR'}], []]
    ids = [0, 1, 2, 3, 4]

    labels = ['[PAD]', '[CLS]', '[SEP]', 'O', 'B-ACTOR', 'I-ACTOR', 'B-YEAR', 'B-TITLE', 'B-GENRE', 'I-GENRE', 'B-DIRECTOR', 'I-DIRECTOR', 'B-SONG', 'I-SONG', 'B-PLOT', 'I-PLOT', 'B-REVIEW', 'B-CHARACTER', 'I-CHARACTER', 'B-RATING', 'B-RATINGS_AVERAGE', 'I-RATINGS_AVERAGE', 'I-TITLE', 'I-RATING', 'B-TRAILER', 'I-TRAILER', 'I-REVIEW', 'I-YEAR']
    dq.set_labels_for_run(labels)
    dq.set_tagging_schema("BIO")
    # dq.log_data_samples(texts=text_inputs, text_token_indices=tokens, ids=ids, gold_spans=gold_spans, split="training")
    # dq.log_data_samples(texts=text_inputs, text_token_indices=tokens, ids=ids, gold_spans=gold_spans, split="validation")
    dq.log_data_samples(texts=text_inputs, text_token_indices=tokens, ids=ids, split="inference", inference_name="hi")

def log_outputs():
    num_classes = 28
    embs = [np.random.rand(119, 768) for _ in range(5)]
    logits= [np.random.rand(119, 28) for _ in range(5)]                                      
    ids= list(range(5))
    for split in ["inference"]:
        dq.log_model_outputs(
            embs=embs, logits=logits, ids=ids, split=split, inference_name="hi"
        )
    
def finish():
    dq.finish()
    
    
def runit():
    log_inputs()
    log_outputs()
    finish()
    
# runit()
# df_train, df_test, df_val = see_results()

📡 Retrieving run from existing project, test-ner-proj
🛰 Connected to project, test-ner-proj, and run, test-ner-run.


/Users/elliottchartock/Code/dataquality/.venv/lib/python3.9/site-packages/dataquality/core/init.py:164: UserWarning: Run: test-ner-proj/test-ner-run already exists! The existing run will get overwritten on call to finish()!
  warnings.warn(


In [5]:
log_inputs()

Logging 5 samples [########################################] 100.00% elapsed time  :     0.00s =  0.0m =  0.0h
 

In [6]:
log_outputs()

In [7]:
finish()

☁️ Uploading Data


inference:   0%|          | 0/1 [00:00<?, ?it/s]

Processing data for upload:   0%|          | 0/1 [00:00<?, ?it/s]

> /Users/elliottchartock/Code/dataquality/.venv/lib/python3.9/site-packages/dataquality/loggers/data_logger/text_ner.py(676)split_dataframe()
    674 
    675         import pdb; pdb.set_trace()
--> 676         prob = df_copy[prob_cols]
    677         emb_cols = (
    678             [NERCols.id.value] if prob_only else [NERCols.id.value, NERCols.emb.value]



ipdb>  n


...

Uploading data to Galileo:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

> /Users/elliottchartock/Code/dataquality/.venv/lib/python3.9/site-packages/dataquality/core/finish.py(61)finish()
     59         )
     60     import pdb; pdb.set_trace()
---> 61     res = api_client.make_request(
     62         RequestType.POST, url=f"{config.api_url}/{Route.jobs}", body=body
     63     )



ipdb>  c


Job inference successfully submitted. Results will be available soon at http://127.0.0.1:3000/insights?projectId=86426353-ba08-4b02-995a-7c62c77f1de9&runId=7d079a21-c1a9-45aa-b9c8-99ec384726a7&split=training&depHigh=1&depLow=0&taskType=2
Waiting for job...


GalileoException: It seems your run failed with status failed, error "Failed background job: Inference is not yet support for NER\n\nTraceback (most recent call last):\n  File \"/Users/elliottchartock/Code/runners/.venv/lib/python3.9/site-packages/runners/servicers/vaex_runner_servicer.py\", line 145, in _run\n    job_fn()\n  File \"/Users/elliottchartock/Code/runners/.venv/lib/python3.9/site-packages/ddtrace/tracer.py\", line 920, in func_wrapper\n    return f(*args, **kwargs)\n  File \"/Users/elliottchartock/Code/runners/.venv/lib/python3.9/site-packages/rungalileo/services/vaex/base_vaex.py\", line 1368, in run\n    self.inference(job_request=job_request, df_dir=tempdir)\n  File \"/Users/elliottchartock/Code/runners/.venv/lib/python3.9/site-packages/rungalileo/services/vaex/text_ner.py\", line 856, in inference\n    raise NotImplementedError(\"Inference is not yet support for NER\")\nNotImplementedError: Inference is not yet support for NER\n"